# Character Level Tokenization

Code taken from [Andrej Karpathy](https://colab.research.google.com/drive/1JMLa53HDuA-i7ZBmqV7ZnA3c_fvtXnx-?usp=sharing).

In [ ]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [ ]:
# We always start with a dataset to train on. Let's download the tiny shakespeare dataset
#!wget https://raw.githubusercontent.com/karpathsy/char-rnn/master/data/tinyshakespeare/input.txt
#!wget https://github.com/cltk/hindi_text_ltrc/blob/master/tulasidaas/Raamacharita_maanasa/1/main.txt

#!wget https://sherlock-holm.es/stories/plain-text/stud.txt

!wget https://raw.githubusercontent.com/cltk/hindi_text_ltrc/master/tulasidaas/Raamacharita_maanasa/1/main.txt

--2024-06-20 15:07:48--  https://raw.githubusercontent.com/cltk/hindi_text_ltrc/master/tulasidaas/Raamacharita_maanasa/1/main.txt
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 516373 (504K) [text/plain]
Saving to: ‘main.txt.2’

main.txt.2          100%[===================>] 504.27K  --.-KB/s    in 0.05s   

2024-06-20 15:07:48 (10.2 MB/s) - ‘main.txt.2’ saved [516373/516373]



In [ ]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

url = "https://raw.githubusercontent.com/cltk/hindi_text_ltrc/master/tulasidaas/Raamacharita_maanasa/1/main.txt"
html = urlopen(url).read()
soup = BeautifulSoup(html, features="html.parser")

# kill all script and style elements
for script in soup(["script", "style"]):
    script.extract()    # rip it out

# get text
text = soup.get_text()
ramayana_text = text
print(type(text))

<class 'str'>


In [ ]:
print("length of dataset in characters: ", len(text))

length of dataset in characters:  198223


In [ ]:
# let's look at the first 1000 characters
print(text[:1000])

जो सुमिरत सिधि होइ गन नायक करिबर बदन।
करउ अनुग्रह सोइ बुद्धि रासि सुभ गुन सदन॥1॥
मूक होइ बाचाल पंगु चढइ गिरिबर गहन।
जासु कृपाँ सो दयाल द्रवउ सकल कलि मल दहन॥2॥
नील सरोरुह स्याम तरुन अरुन बारिज नयन।
करउ सो मम उर धाम सदा छीरसागर सयन॥3॥
कुंद इंदु सम देह उमा रमन करुना अयन।
जाहि दीन पर नेह करउ कृपा मर्दन मयन॥4॥
बंदउ गुरु पद कंज कृपा सिंधु नररूप हरि।
महामोह तम पुंज जासु बचन रबि कर निकर॥5॥
बंदउ गुरु पद पदुम परागा। सुरुचि सुबास सरस अनुरागा॥
अमिय मूरिमय चूरन चारू। समन सकल भव रुज परिवारू॥
सुकृति संभु तन बिमल बिभूती। मंजुल मंगल मोद प्रसूती॥
जन मन मंजु मुकुर मल हरनी। किएँ तिलक गुन गन बस करनी॥
श्रीगुर पद नख मनि गन जोती। सुमिरत दिब्य द्रृष्टि हियँ होती॥
दलन मोह तम सो सप्रकासू। बड़े भाग उर आवइ जासू॥
उघरहिं बिमल बिलोचन ही के। मिटहिं दोष दुख भव रजनी के॥
सूझहिं राम चरित मनि मानिक। गुपुत प्रगट जहँ जो जेहि खानिक॥
दो0-जथा सुअंजन अंजि दृग साधक सिद्ध सुजान।
कौतुक देखत सैल बन भूतल भूरि निधान॥1॥

एहि महँ रघुपति नाम उदारा। अति पावन पुरान श्रुति सारा॥
मंगल भवन अमंगल हारी। उमा सहित जेहि जपत पुरारी॥
भनिति बिचित्र स

In [ ]:
# here are all the unique characters that occur in this text
chars = sorted(list(set(text)))
vocab_size = len(chars)
print(''.join(chars))
print(vocab_size)


 (),-0123456789ûँंःअआइईउऊएऐओऔकखगघङचछजझटठडढणतथदधनपफबभमयरलवशषसह़ऽािीुूृेैोौ्।॥०१२३४५६७८९
87


In [ ]:
# create a mapping from characters to integers
stoi = { ch:i for i,ch in enumerate(chars) }
itos = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [stoi[c] for c in s] # encoder: take a string, output a list of integers
decode = lambda l: ''.join([itos[i] for i in l]) # decoder: take a list of integers, output a string

print(encode("एहि महँ रघुपति "))
print(decode(encode("एहि महँ रघुपति")))

[26, 61, 65, 1, 53, 61, 17, 1, 55, 33, 67, 49, 44, 65, 1]
एहि महँ रघुपति


In [ ]:
# let's now encode the entire text dataset and store it into a torch.Tensor
import torch # we use PyTorch: https://pytorch.org
data = torch.tensor(encode(text), dtype=torch.long)
print(data.shape, data.dtype)
print(data[:1000]) # the 1000 characters we looked at earier will to the LLM look like this

torch.Size([198223]) torch.int64
tensor([37, 72,  1, 60, 67, 53, 65, 55, 44,  1, 60, 65, 47, 65,  1, 61, 72, 22,
         1, 32, 48,  1, 48, 64, 54, 30,  1, 30, 55, 65, 51, 55,  1, 51, 46, 48,
        75,  0, 30, 55, 24,  1, 20, 48, 67, 32, 74, 55, 61,  1, 60, 72, 22,  1,
        51, 67, 46, 74, 47, 65,  1, 55, 64, 60, 65,  1, 60, 67, 52,  1, 32, 67,
        48,  1, 60, 46, 48, 76,  7, 76,  0, 53, 68, 30,  1, 61, 72, 22,  1, 51,
        64, 35, 64, 56,  1, 49, 18, 32, 67,  1, 35, 42, 22,  1, 32, 65, 55, 65,
        51, 55,  1, 32, 61, 48, 75,  0, 37, 64, 60, 67,  1, 30, 69, 49, 64, 17,
         1, 60, 72,  1, 46, 54, 64, 56,  1, 46, 74, 55, 57, 24,  1, 60, 30, 56,
         1, 30, 56, 65,  1, 53, 56,  1, 46, 61, 48, 76,  8, 76,  0, 48, 66, 56,
         1, 60, 55, 72, 55, 67, 61,  1, 60, 74, 54, 64, 53,  1, 44, 55, 67, 48,
         1, 20, 55, 67, 48,  1, 51, 64, 55, 65, 37,  1, 48, 54, 48, 75,  0, 30,
        55, 24,  1, 60, 72,  1, 53, 53,  1, 24, 55,  1, 47, 64, 53,  1, 60, 46,
       

In [34]:
data = torch.tensor(encode("कौन है रघुपति"), dtype=torch.long)
print(data.shape, data.dtype)
print(data)

torch.Size([8]) torch.int64
tensor([286, 357, 264, 325, 337, 997, 282, 260])


In [35]:
text = "नाम जीहँ जपि जागहिं जोगी। बिरति बिरंचि प्रपंच बियोगी॥"
tokens = text.encode("utf-8") # raw bytes
tokens = list(map(int, tokens)) # convert to a list of integers in range 0..255 for convenience
print('---')
print(text)
print("length:", len(text))
print('---')
print(tokens)
print("length:", len(tokens))

---
नाम जीहँ जपि जागहिं जोगी। बिरति बिरंचि प्रपंच बियोगी॥
length: 53
---
[224, 164, 168, 224, 164, 190, 224, 164, 174, 32, 224, 164, 156, 224, 165, 128, 224, 164, 185, 224, 164, 129, 32, 224, 164, 156, 224, 164, 170, 224, 164, 191, 32, 224, 164, 156, 224, 164, 190, 224, 164, 151, 224, 164, 185, 224, 164, 191, 224, 164, 130, 32, 224, 164, 156, 224, 165, 139, 224, 164, 151, 224, 165, 128, 224, 165, 164, 32, 224, 164, 172, 224, 164, 191, 224, 164, 176, 224, 164, 164, 224, 164, 191, 32, 224, 164, 172, 224, 164, 191, 224, 164, 176, 224, 164, 130, 224, 164, 154, 224, 164, 191, 32, 224, 164, 170, 224, 165, 141, 224, 164, 176, 224, 164, 170, 224, 164, 130, 224, 164, 154, 32, 224, 164, 172, 224, 164, 191, 224, 164, 175, 224, 165, 139, 224, 164, 151, 224, 165, 128, 224, 165, 165]
length: 143


Let's find the pair of bytes that occur most commonly and then replace them

In [41]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]): # Pythonic way to iterate consecutive elements
        counts[pair] = counts.get(pair, 0) + 1
    return counts

stats = get_stats(tokens)
print(stats)
print(sorted(((v,k) for k,v in stats.items()), reverse=True))

{(65, 32): 8, (32, 80): 10, (80, 114): 1, (114, 111): 62, (111, 103): 23, (103, 114): 25, (114, 97): 106, (97, 109): 47, (109, 109): 20, (109, 101): 69, (101, 114): 294, (114, 226): 8, (226, 128): 254, (128, 153): 64, (153, 115): 29, (115, 32): 424, (32, 73): 36, (73, 110): 15, (110, 116): 176, (116, 114): 50, (111, 100): 183, (100, 117): 8, (117, 99): 19, (99, 116): 68, (116, 105): 168, (105, 111): 76, (111, 110): 173, (110, 32): 241, (32, 116): 405, (116, 111): 124, (111, 32): 137, (32, 85): 127, (85, 110): 60, (110, 105): 98, (105, 99): 146, (99, 111): 290, (100, 101): 232, (101, 32): 646, (32, 77): 16, (77, 97): 7, (97, 114): 181, (114, 99): 11, (99, 104): 89, (104, 32): 76, (32, 51): 6, (51, 44): 2, (44, 32): 243, (32, 50): 6, (50, 48): 1, (48, 49): 7, (49, 55): 2, (55, 32): 5, (32, 194): 2, (194, 183): 2, (183, 32): 2, (32, 67): 22, (67, 111): 17, (100, 105): 88, (105, 110): 446, (110, 103): 200, (103, 32): 111, (50, 50): 1, (50, 32): 14, (111, 109): 114, (101, 110): 174, (116, 1

In [42]:
top_pair = max(stats, key=stats.get)
top_pair

(101, 32)

In [43]:
from IPython.display import HTML, display

def set_css():
  display(HTML('''
  <style>
    pre {
        white-space: pre-wrap;
    }
  </style>
  '''))
get_ipython().events.register('pre_run_cell', set_css)

In [44]:
def merge(ids, pair, idx):
  # in the list of ints (ids), replace all consecutive occurences of pair with the new token idx
  newids = []
  i = 0
  while i < len(ids):
    # if we are not at the very last position AND the pair matches, replace it
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1
  return newids

print(merge([5, 6, 6, 7, 9, 1], (6, 7), 99))

tokens2 = merge(tokens, top_pair, 256)


print(tokens2)
print("length:", len(tokens2), len(tokens))

[5, 6, 99, 9, 1]
[65, 32, 80, 114, 111, 103, 114, 97, 109, 109, 101, 114, 226, 128, 153, 115, 32, 73, 110, 116, 114, 111, 100, 117, 99, 116, 105, 111, 110, 32, 116, 111, 32, 85, 110, 105, 99, 111, 100, 256, 77, 97, 114, 99, 104, 32, 51, 44, 32, 50, 48, 49, 55, 32, 194, 183, 32, 67, 111, 100, 105, 110, 103, 32, 194, 183, 32, 50, 50, 32, 67, 111, 109, 109, 101, 110, 116, 115, 32, 32, 239, 188, 181, 239, 189, 142, 239, 189, 137, 239, 189, 131, 239, 189, 143, 239, 189, 132, 239, 189, 133, 33, 32, 240, 159, 133, 164, 240, 159, 133, 157, 240, 159, 133, 152, 240, 159, 133, 146, 240, 159, 133, 158, 240, 159, 133, 147, 240, 159, 133, 148, 226, 128, 189, 32, 240, 159, 135, 186, 226, 128, 140, 240, 159, 135, 179, 226, 128, 140, 240, 159, 135, 174, 226, 128, 140, 240, 159, 135, 168, 226, 128, 140, 240, 159, 135, 180, 226, 128, 140, 240, 159, 135, 169, 226, 128, 140, 240, 159, 135, 170, 33, 32, 240, 159, 152, 132, 32, 84, 104, 256, 118, 101, 114, 121, 32, 110, 97, 109, 256, 115, 116, 114, 105, 107,

In [46]:
tokens = text.encode("utf-8") # raw bytes
tokens = list(map(int, tokens)) # convert to a list of integers in range 0..255 for convenience

In [47]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
  newids = []
  i = 0
  while i < len(ids):
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1
  return newids

# ---
vocab_size = 276 # the desired final vocabulary size
num_merges = vocab_size - 256
ids = list(tokens) # copy so we don't destroy the original list

merges = {} # (int, int) -> int
for i in range(num_merges):
  stats = get_stats(ids)
  pair = max(stats, key=stats.get)
  idx = 256 + i
  print(f"merging {pair} into a new token {idx}")
  ids = merge(ids, pair, idx)
  merges[pair] = idx

merging (101, 32) into a new token 256
merging (105, 110) into a new token 257
merging (115, 32) into a new token 258
merging (116, 104) into a new token 259
merging (101, 114) into a new token 260
merging (99, 111) into a new token 261
merging (116, 32) into a new token 262
merging (226, 128) into a new token 263
merging (44, 32) into a new token 264
merging (97, 110) into a new token 265
merging (111, 114) into a new token 266
merging (100, 32) into a new token 267
merging (97, 114) into a new token 268
merging (101, 110) into a new token 269
merging (257, 103) into a new token 270
merging (261, 100) into a new token 271
merging (121, 32) into a new token 272
merging (46, 32) into a new token 273
merging (97, 108) into a new token 274
merging (259, 256) into a new token 275


In [48]:
print("tokens length:", len(tokens))
print("ids length:", len(ids))
print(f"compression ratio: {len(tokens) / len(ids):.2f}X")

tokens length: 24597
ids length: 19438
compression ratio: 1.27X


In [49]:
def get_stats(ids):
    counts = {}
    for pair in zip(ids, ids[1:]):
        counts[pair] = counts.get(pair, 0) + 1
    return counts

def merge(ids, pair, idx):
  newids = []
  i = 0
  while i < len(ids):
    if i < len(ids) - 1 and ids[i] == pair[0] and ids[i+1] == pair[1]:
      newids.append(idx)
      i += 2
    else:
      newids.append(ids[i])
      i += 1
  return newids

# ---
vocab_size = 1000 # the desired final vocabulary size
num_merges = vocab_size - 256
ids = list(tokens) # copy so we don't destroy the original list

merges = {} # (int, int) -> int
for i in range(num_merges):
  stats = get_stats(ids)
  pair = max(stats, key=stats.get)
  idx = 256 + i
  # print(f"merging {pair} into a new token {idx}")
  ids = merge(ids, pair, idx)
  merges[pair] = idx

print("tokens length:", len(tokens))
print("ids length:", len(ids))
print(f"compression ratio: {len(tokens) / len(ids):.2f}X")

tokens length: 24597
ids length: 8120
compression ratio: 3.03X


### Decoding


In [50]:
vocab = {idx: bytes([idx]) for idx in range(256)}
for (p0, p1), idx in merges.items():
    vocab[idx] = vocab[p0] + vocab[p1]

def decode(ids):
  # given ids (list of integers), return Python string
  tokens = b"".join(vocab[idx] for idx in ids)
  text = tokens.decode("utf-8", errors="replace")
  return text

print(decode([128]))

�


### Encoding

The other way around: Given a string, what are the tokens?


In [51]:
merges

{(101, 32): 256,
 (105, 110): 257,
 (115, 32): 258,
 (116, 104): 259,
 (101, 114): 260,
 (99, 111): 261,
 (116, 32): 262,
 (226, 128): 263,
 (44, 32): 264,
 (97, 110): 265,
 (111, 114): 266,
 (100, 32): 267,
 (97, 114): 268,
 (101, 110): 269,
 (257, 103): 270,
 (261, 100): 271,
 (121, 32): 272,
 (46, 32): 273,
 (97, 108): 274,
 (259, 256): 275,
 (111, 110): 276,
 (111, 32): 277,
 (116, 105): 278,
 (97, 99): 279,
 (101, 115): 280,
 (111, 102): 281,
 (271, 256): 282,
 (112, 111): 283,
 (270, 32): 284,
 (281, 32): 285,
 (101, 267): 286,
 (116, 260): 287,
 (116, 277): 288,
 (257, 32): 289,
 (97, 32): 290,
 (266, 32): 291,
 (115, 116): 292,
 (111, 117): 293,
 (99, 104): 294,
 (120, 120): 295,
 (101, 120): 296,
 (101, 258): 297,
 (263, 156): 298,
 (263, 157): 299,
 (261, 109): 300,
 (265, 267): 301,
 (112, 108): 302,
 (105, 116): 303,
 (283, 257): 304,
 (114, 101): 305,
 (263, 153): 306,
 (110, 105): 307,
 (274, 108): 308,
 (97, 116): 309,
 (260, 32): 310,
 (105, 258): 311,
 (282, 304): 312,

In [53]:
def encode(text):
  # given a string, return list of integers (the tokens)
  tokens = list(text.encode("utf-8"))
  while len(tokens) >= 2:
    stats = get_stats(tokens)
    pair = min(stats, key=lambda p: merges.get(p, float("inf")))
    if pair not in merges:
      break # nothing else can be merged
    idx = merges[pair]
    tokens = merge(tokens, pair, idx)
  return tokens

print(encode("र"))

[224, 164, 176]


In [54]:
print(decode(encode("र")))

र


In [55]:
text2 = decode(encode(text))
print(text2 == text)

True
